In [2]:
import serial
import serial.tools.list_ports
import time
import os
import cv2
import csv
import numpy as np
from datetime import datetime


In [4]:
BAUD_RATE = 9600
ser = serial.Serial('/dev/ttyACM1', BAUD_RATE)

def connect():
    # connect to Arduino
    ports = list(serial.tools.list_ports.comports())  # Create a list with info for all devices
    connected = False
    # Look for the word 'Arduino' in device info, store this port
    for p in ports:
        if "Arduino" in p[1]:
            connected = True
            port_arduino = p[0]
            print("Connected to Arduino on port: ", port_arduino)
            break
    if not connected:
        print("No Arduino Found")

    ser = serial.Serial(port_arduino, BAUD_RATE)
    return ser


def send_cmd(ser,cmd):
    """
    send string command to Arduino

    :param cmd: (string) command to send
    :return:
    """
    # print(cmd)
    for c in cmd:
        # self.ser.write(c.encode())
        ser.write(c.encode())
        time.sleep(.01)
    ser.write('\r'.encode())
    time.sleep(.01)

def clear_buff(ser):
    while ser.in_waiting:
        ser.read()

def read_char(ser):
    # use Serial.print() on Arduino
    c = ser.read().decode()
    return c


In [5]:
ser = connect()

Connected to Arduino on port:  /dev/ttyACM1


In [6]:
clear_buff(ser)
send_cmd(ser, ' ')
ser.write(b' \n') 
clear_buff(ser)

In [7]:
send_cmd(ser, '2 250 0')

In [ ]:
send_cmd(ser, '2 200 350')

In [8]:
def send_cmd_new(ser, mode, start_x, y_range, cam_int, cam_top):
    count = 0
    while start_x > 0:
        for y in y_range:
            time.sleep(2)
            cmd = f'{mode} {start_x} {y}'
            count += 1
            direction = 'left'
            send_single_cmd(ser, cmd, count, direction, start_x, y, cam_int, cam_top)
            # interior_camprocessor(cam_no_int, count)
            # coordinates(direction, start_x, y, cam_no_top, count)
            # start_x -= 1
            reading = ser.readline().decode()
            if reading != '':
                print(reading)
            # time.sleep(2)

        start_x -= 1

        for y in reversed(y_range):
            cmd = f'{mode} {start_x} {y}'
            count += 1
            direction = 'right'
            send_single_cmd(ser, cmd, count, direction, start_x, y, cam_int, cam_top)
            # interior_camprocessor(cam_no_int, count)
            # coordinates(direction_now, start_x, y, cam_no_top, count)
            # time.sleep(2)

        start_x -= 1

def send_single_cmd(ser, cmd, count, direction, start_x, y, cam_int, cam_top):
    ser.write(cmd.encode())
    ser.write('\r'.encode())
    # time.sleep(0.01)
    # time.sleep(2)
    interior_camprocessor(cam_int, count)
    coordinates(direction, start_x, y, cam_top, count)
    time.sleep(0.01)
    read_char(ser)  


In [ ]:
# if __name__ == "__main__":
#     mode = '2'
#     start_x = 200
#     y_range = range(0, 350, 10)
#     send_cmd(ser, mode, start_x, y_range)

In [18]:

def coordinates(direction_now, x_step_now, y_step_now, cam_no, ith_number):
    camera_matrix = np.array([[682.694, 0, 573.5379], [0, 675.82, 367.535], [0, 0, 1]])
    dist_coeffs = np.array([-0.119, 0.1449, 0.0019, -0.00689, -0.1701])

    aruco_dict = cv2.aruco.getPredefinedDictionary(cv2.aruco.DICT_4X4_50)
    marker_size = 2.6  # or 2.61

        # Function to calculate marker pose
    def estimate_marker_pose(marker_corners):
        rvecs, tvecs, _ = cv2.aruco.estimatePoseSingleMarkers(marker_corners, marker_size, camera_matrix, dist_coeffs)
        return rvecs, tvecs
    cap = cv2.VideoCapture(cam_no)
    with open(output_file, mode='a', newline='') as file:
        writer = csv.writer(file)
        ret, frame = cap.read()  # Capture a single frame
        cap.release()  # Release the capture device

        if ret:
            gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            corners, ids, _ = cv2.aruco.detectMarkers(gray, aruco_dict)

            if ids is not None:
                rvecs, tvecs = estimate_marker_pose(corners)
                # rvecs, tvecs = cv2.aruco.estimatePoseSingleMarkers(corners, marker_size, camera_matrix, dist_coeffs)
                values = {}
                for i in range(len(ids)):
                    cv2.aruco.drawDetectedMarkers(frame, corners)
                    cv2.aruco.drawAxis(frame, camera_matrix, dist_coeffs, rvecs[i], tvecs[i], marker_size * 0.5)
                    # x, y, z = tvecs[0][0] - tvecs[1][0]
                    if ids[i][0] == 3: # that is the robot wrt top cam
                        variable_name = "var_0"
                        values[variable_name] = tvecs[i][0]
                    elif ids[i][0] == 4:
                        variable_name = "var_1"
                        values[variable_name] = tvecs[i][0]
                    # variable_name = "var_" + str(i)
                    # values[variable_name] = tvecs[i][0]
                    print("Marker ID:", ids[i][0])
                    # print("Position (x, y):", x, y)
                print(values)
                if len(values) == 2:
                    x, y, z = values['var_0'] - values['var_1'] #check obj wrt R is (what minus what)
                else:
                    x = y = z = float("NaN")
                timestamp = datetime.now().strftime("%Y-%m-%d %H-%M-%S.%f")
                writer.writerow([ith_number, timestamp, x, y, direction_now, x_step_now, y_step_now])
                    # row_data = [ith_number, timestamp, x, y, direction_now, x_step_now, y_step_now]

                    # Open the output file in append mode
                    # with open(output_file, mode='a', newline='') as file:
                    #     writer = csv.writer(file)
                    #     writer.writerow(row_data)

            cv2.imshow("ArUco Marker Detection", frame)
        # cv2.waitKey(0)

    cv2.destroyAllWindows()


In [ ]:
# def coordinates(direction_now, x_step_now, y_step_now, cam_no, ith_number):
#     camera_matrix = np.array([[682.694, 0, 573.5379], [0, 675.82, 367.535], [0, 0, 1]])
#     dist_coeffs = np.array([-0.119, 0.1449, 0.0019, -0.00689, -0.1701])

#     aruco_dict = cv2.aruco.getPredefinedDictionary(cv2.aruco.DICT_4X4_50)
#     marker_size = 2.6  # or 2.61
    
#     # output_file = 'data_collection.csv'
#     def estimate_marker_pose(marker_corners):
#         rvecs, tvecs, _ = cv2.aruco.estimatePoseSingleMarkers(marker_corners, marker_size, camera_matrix, dist_coeffs)
#         return rvecs, tvecs

#     # cap = cv2.VideoCapture(6)
#     cap = cv2.VideoCapture(cam_no)
#     # Open the output file for writing
#     with open(output_file, mode='w', newline='') as file:
#         writer = csv.writer(file)
#         # Updated header row to include direction and x-step columns
#         writer.writerow(['Serial_no.','Timestamp', 'x_actual', 'y_actual', 'direction', 'x_step', 'y_step'])

#         direction = direction_now
#         x_step = x_step_now
#         y_step = y_step_now

#         while True:
#             ret, frame = cap.read()
#             if ret:
#                 gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY) # Convert to grayscale
#                 corners, ids, _ = cv2.aruco.detectMarkers(gray, aruco_dict) # Detect ArUco markers
#                 # If markers are detected
#                 if ids is not None:
#                     rvecs, tvecs = estimate_marker_pose(corners)
#                     for i in range(len(ids)):
#                         cv2.aruco.drawDetectedMarkers(frame, corners)  # Draw marker outline
#                         cv2.aruco.drawAxis(frame, camera_matrix, dist_coeffs, rvecs[i], tvecs[i], marker_size * 0.5) # Draw axis for each marker
#                         # Get real-world coordinates
#                         # x, y, z = tvecs[i][0]
#                         # x_CR - x_CO = x_RO and same for y-axis
#                         x, y, z = tvecs[0][0] - tvecs[1][0]
#                         print("Marker ID:", ids[i][0])
#                         print("Position (x, y):", x, y)
#                         # Get the current timestamp
#                         timestamp = datetime.now().strftime("%Y-%m-%d %H-%M-%S.%f")
#                         writer.writerow([ith_number, timestamp, x, y, direction, x_step, y_step]) ######Also add a number as identifier not just timestamp

#                 cv2.imshow("ArUco Marker Detection", frame)

#             if cv2.waitKey(1) & 0xFF == ord('q'):
#                 break

#     cap.release()
#     cv2.destroyAllWindows()


In [12]:

def interior_camprocessor(cam_no, ith_number):
    # cap = cv2.VideoCapture(8)
    cap = cv2.VideoCapture(cam_no)
    interval = 0.04  # interval in seconds (25 frames per second)(25Hz)
    counter = 0  # initialize the counter

    # output_folder = "output"

    # Create the output folder if it doesn't exist
    os.makedirs(output_folder, exist_ok=True)

    # while True:
    ret, frame = cap.read()
    if ret:
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S.%f")
        filename = os.path.join(output_folder, f"{ith_number}_{timestamp}.jpg")
        cv2.imwrite(filename, frame)
        
        counter += 1  # increment the counter variable
    time.sleep(interval)

    cap.release()
    # break


In [3]:
def cam_ports():
    all_camera_idx_available = []

    for camera_idx in range(30):
        cap = cv2.VideoCapture(camera_idx)
        if cap.isOpened():
            print(f'Camera index available: {camera_idx}')
            all_camera_idx_available.append(camera_idx)
            cap.release()
    print(all_camera_idx_available)

In [4]:
cam_ports()

Camera index available: 0


[ WARN:0] global ./modules/videoio/src/cap_gstreamer.cpp (1100) open OpenCV | GStreamer warning: Cannot query video position: status=0, value=-1, duration=-1
[ WARN:0] global ./modules/videoio/src/cap_gstreamer.cpp (2075) handleMessage OpenCV | GStreamer warning: Embedded video playback halted; module v4l2src2 reported: Device '/dev/video1' is not a capture device.
[ WARN:0] global ./modules/videoio/src/cap_gstreamer.cpp (1053) open OpenCV | GStreamer warning: unable to start pipeline
[ WARN:0] global ./modules/videoio/src/cap_gstreamer.cpp (616) isPipelinePlaying OpenCV | GStreamer warning: GStreamer: pipeline have not been created
[ WARN:0] global ./modules/videoio/src/cap_v4l.cpp (890) open VIDEOIO(V4L2:/dev/video1): can't open camera by index
[ WARN:0] global ./modules/videoio/src/cap_gstreamer.cpp (2075) handleMessage OpenCV | GStreamer warning: Embedded video playback halted; module v4l2src3 reported: Internal data stream error.
[ WARN:0] global ./modules/videoio/src/cap_gstreame

Camera index available: 4


[ WARN:0] global ./modules/videoio/src/cap_gstreamer.cpp (1100) open OpenCV | GStreamer warning: Cannot query video position: status=0, value=-1, duration=-1


Camera index available: 6
Camera index available: 7


[ WARN:0] global ./modules/videoio/src/cap_gstreamer.cpp (1100) open OpenCV | GStreamer warning: Cannot query video position: status=0, value=-1, duration=-1
[ WARN:0] global ./modules/videoio/src/cap_gstreamer.cpp (2075) handleMessage OpenCV | GStreamer warning: Embedded video playback halted; module v4l2src9 reported: Device '/dev/video8' is not a capture device.
[ WARN:0] global ./modules/videoio/src/cap_gstreamer.cpp (1053) open OpenCV | GStreamer warning: unable to start pipeline
[ WARN:0] global ./modules/videoio/src/cap_gstreamer.cpp (616) isPipelinePlaying OpenCV | GStreamer warning: GStreamer: pipeline have not been created
[ WARN:0] global ./modules/videoio/src/cap_v4l.cpp (890) open VIDEOIO(V4L2:/dev/video8): can't open camera by index
[ WARN:0] global ./modules/videoio/src/cap_gstreamer.cpp (2075) handleMessage OpenCV | GStreamer warning: Embedded video playback halted; module v4l2src10 reported: Internal data stream error.
[ WARN:0] global ./modules/videoio/src/cap_gstream

[0, 4, 6, 7]


[ WARN:0] global ./modules/videoio/src/cap_gstreamer.cpp (2075) handleMessage OpenCV | GStreamer warning: Embedded video playback halted; module v4l2src28 reported: Cannot identify device '/dev/video27'.
[ WARN:0] global ./modules/videoio/src/cap_gstreamer.cpp (1053) open OpenCV | GStreamer warning: unable to start pipeline
[ WARN:0] global ./modules/videoio/src/cap_gstreamer.cpp (616) isPipelinePlaying OpenCV | GStreamer warning: GStreamer: pipeline have not been created
[ WARN:0] global ./modules/videoio/src/cap_v4l.cpp (890) open VIDEOIO(V4L2:/dev/video27): can't open camera by index
[ WARN:0] global ./modules/videoio/src/cap_gstreamer.cpp (2075) handleMessage OpenCV | GStreamer warning: Embedded video playback halted; module v4l2src29 reported: Cannot identify device '/dev/video28'.
[ WARN:0] global ./modules/videoio/src/cap_gstreamer.cpp (1053) open OpenCV | GStreamer warning: unable to start pipeline
[ WARN:0] global ./modules/videoio/src/cap_gstreamer.cpp (616) isPipelinePlaying

In [4]:
import cv2
from cv2 import aruco
def show_marker():
    # ### Overlay marker ID and video ###
    dict_aruco = aruco.Dictionary_get(aruco.DICT_4X4_50)
    parameters = aruco.DetectorParameters_create()

    cap = cv2.VideoCapture(6)

    try:
        while True:
            ret, frame = cap.read()
            gray = cv2.cvtColor(frame, cv2.COLOR_RGB2GRAY)

            corners, ids, rejectedImgPoints = aruco.detectMarkers(gray, dict_aruco, parameters=parameters)

            frame_markers = aruco.drawDetectedMarkers(frame.copy(), corners, ids)
            cv2.imshow('frame', frame_markers)
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
        cv2.destroyWindow('frame')
        cap.release()
    except KeyboardInterrupt:
        cv2.destroyWindow('frame')
        cap.release()


In [5]:
show_marker()

[ WARN:0] global ./modules/videoio/src/cap_gstreamer.cpp (1100) open OpenCV | GStreamer warning: Cannot query video position: status=0, value=-1, duration=-1


: 

In [19]:
if __name__ == "__main__":
    mode = '2'
    start_x = 200
    cam_no_int = 11#14
    cam_no_top = 13#6
    y_range = range(0, 350, 10)
    output_file = 'data_collection4.csv'
    output_folder = "output_images_new"
    send_cmd_new(ser, mode, start_x, y_range, cam_no_int, cam_no_top)

[ WARN:0] global ./modules/videoio/src/cap_gstreamer.cpp (1100) open OpenCV | GStreamer warning: Cannot query video position: status=0, value=-1, duration=-1
[ WARN:0] global ./modules/videoio/src/cap_gstreamer.cpp (1100) open OpenCV | GStreamer warning: Cannot query video position: status=0, value=-1, duration=-1


Marker ID: 3
{'var_0': array([-17.3329047 , -15.84969631,  35.3063755 ])}
00



[ WARN:0] global ./modules/videoio/src/cap_gstreamer.cpp (1100) open OpenCV | GStreamer warning: Cannot query video position: status=0, value=-1, duration=-1
[ WARN:0] global ./modules/videoio/src/cap_gstreamer.cpp (1100) open OpenCV | GStreamer warning: Cannot query video position: status=0, value=-1, duration=-1


Marker ID: 0
Marker ID: 3
{'var_0': array([-17.3329047 , -15.84969631,  35.3063755 ])}
 target:



[ WARN:0] global ./modules/videoio/src/cap_gstreamer.cpp (1100) open OpenCV | GStreamer warning: Cannot query video position: status=0, value=-1, duration=-1
[ WARN:0] global ./modules/videoio/src/cap_gstreamer.cpp (1100) open OpenCV | GStreamer warning: Cannot query video position: status=0, value=-1, duration=-1


...

[ WARN:0] global ./modules/videoio/src/cap_gstreamer.cpp (1100) open OpenCV | GStreamer warning: Cannot query video position: status=0, value=-1, duration=-1
[ WARN:0] global ./modules/videoio/src/cap_gstreamer.cpp (1100) open OpenCV | GStreamer warning: Cannot query video position: status=0, value=-1, duration=-1


Marker ID: 4
Marker ID: 0
Marker ID: 3
{'var_1': array([1.62747852e-02, 2.02745492e+01, 5.39114529e+01]), 'var_0': array([-17.3329047 , -15.84969631,  35.3063755 ])}
 target:



[ WARN:0] global ./modules/videoio/src/cap_gstreamer.cpp (1100) open OpenCV | GStreamer warning: Cannot query video position: status=0, value=-1, duration=-1
[ WARN:0] global ./modules/videoio/src/cap_gstreamer.cpp (1100) open OpenCV | GStreamer warning: Cannot query video position: status=0, value=-1, duration=-1


Marker ID: 4
Marker ID: 0
Marker ID: 3
{'var_1': array([1.62747852e-02, 2.02745492e+01, 5.39114529e+01]), 'var_0': array([-17.3329047 , -15.84969631,  35.3063755 ])}




KeyboardInterrupt: 